In [1]:
import pandas as pd

# here i am loading in the parsed comments cleand for the 19
topics_df = pd.read_csv("parsed_comments_cleaned.csv", keep_default_na=False, na_values=[])

# Preview the first few rows
topics_df.head()

,Course Code,Section Code,Instructor,Semester,Comment Type,Comment Text
0,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,N/A
1,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,ok
2,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,n/a
3,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,Good
4,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,Good


Preprocess the comments

In [2]:
import re

def clean_text(text):
	text = str(text).lower()
	text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
	text = re.sub(r"\s+", " ", text).strip()
	return text
# Here i am going over the cleaned comments to see their distribution
topics_df["Cleaned Comment"] = topics_df["Comment Text"].apply(clean_text)
# Then here we are viewing the distribution
topics_df["Cleaned Comment"].value_counts(dropna=False)


Cleaned Comment
good                                                                                                                                                                                                                                                                                                                                                87
na                                                                                                                                                                                                                                                                                                                                                  83
none                                                                                                                                                                                                                                                                                                      

In [3]:
# Ok so clearly we have white spaces so we need to drop those
topics_df = topics_df[topics_df["Cleaned Comment"].notna()]
topics_df = topics_df[topics_df["Cleaned Comment"] != ""]
topics_df["Cleaned Comment"].value_counts(dropna=False)


Cleaned Comment
good                                                                                                                                                                                                                                                                                                                                                87
na                                                                                                                                                                                                                                                                                                                                                  83
none                                                                                                                                                                                                                                                                                                      

In [4]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text).strip()
    return text

topics_df["Cleaned Comment"] = topics_df["Comment Text"].apply(clean_text)
topics_df["Cleaned Comment"].head()



0      na
1      ok
2      na
3    good
4    good
Name: Cleaned Comment, dtype: object

Embed the comment

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(topics_df["Cleaned Comment"].tolist(), show_progress_bar=True)

C:\Users\Admin\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\Admin\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\Admin\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please 

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Running BERTopic

In [6]:
from bertopic import BERTopic

topic_model = BERTopic()
topics, _ = topic_model.fit_transform(topics_df["Cleaned Comment"].tolist(), embeddings)


# Here i am viewing the topics
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,0,91,0_good_poor_hard_but,"[good, poor, hard, but, , , , , , ]","[good, good, good]"
1,1,84,1_na_time_on_,"[na, time, on, , , , , , , ]","[na, na, na]"
2,2,62,2_none_the_and_class,"[none, the, and, class, of, in, me, outside, i...","[none, none, none]"
3,3,17,3_ok_nil_fair_,"[ok, nil, fair, , , , , , , ]","[ok, ok, ok]"
4,4,13,4_excellent_amazing_done_great,"[excellent, amazing, done, great, well, , , , , ]","[excellent, excellent, excellent]"


Here we are going to see the representatives per topic

In [7]:
for topic_id in topic_model.get_topic_info().head(10)["Topic"]:
    print(f"\nTopic {topic_id}:")
    print(topic_model.get_topic(topic_id))
    print("Representative comments:")
    print(topic_model.get_representative_docs()[topic_id][:3])


Topic 0:
[('good', np.float64(0.6279041508191153)), ('poor', np.float64(0.10904474895072412)), ('hard', np.float64(0.04803087617170215)), ('but', np.float64(0.04803087617170215)), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
Representative comments:
['good', 'good', 'good']

Topic 1:
[('na', np.float64(0.6709376927838888)), ('time', np.float64(0.051986360091724686)), ('on', np.float64(0.03609474041333667)), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
Representative comments:
['na', 'na', 'na']

Topic 2:
[('none', np.float64(0.23894658364666585)), ('the', np.float64(0.10772832446868906)), ('and', np.float64(0.078220699376218)), ('class', np.float64(0.06933180112185638)), ('of', np.float64(0.05957384340065275)), ('in', np.float64(0.04869959661877)), ('me', np.float64(0.04869959661877)), ('outside', np.float64(0.04869959661877)), ('it', np.float64(0.04869959661877)), ('knowledge', np.float64(0.04869959661877)

ok so we have different clusters existing here for the na, the ok, the nuetral themes and then actual insightfull feedback. But also mixed clustering that needs to be cleaned up. So lets start to make the taxonomy, label topics, remove alot of filler words like the,and... , and we just add levels of complexity

# Here we will be handling filler words

In [8]:
# Here we are dropping the filler words 
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\s]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

In [9]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\s]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

# Taxonomy creation

In [10]:
taxonomy = {
    "Teaching Delivery": [
        "clarity", "pace", "examples", "engagement", "interactive", "support", "explains", "class"
    ],
    "Course Materials": [
        "slides", "materials", "readings", "upload", "blackboard", "resources", "course text"
    ],
    "Assessment & Feedback": [
        "grading", "rubric", "feedback", "assignments", "marks", "evaluation"
    ],
    "Learning Experience": [
        "real life", "application", "knowledge", "practice", "peer", "collaboration"
    ],
    "Logistics & Operations": [
        "schedule", "time", "office hours", "appointment", "equipment", "facilities"
    ]
}

# Atleast now with the taxonomy we can classify the topics better

# Mapping the topics

In [11]:
topic_labels = {
    0: "General sentiment (mixed)",
    1: "No response (availability)",
    2: "No response (generic)",
    3: "No response (time-related)",
    4: "Positive sentiment (teaching quality)",
    5: "Teaching Delivery – clarity & engagement",
    6: "Neutral sentiment (atomic)"
}
# And these are basic topics we will be using

# Applying the labels to the data frame

In [12]:
# Add taxonomy labels
# Ensure the dataframe has the topic assignments from BERTopic
# 'topics' is the list of topic ids returned by topic_model.fit_transform
topics_df["Topic"] = topics

# Map numeric topic ids to labels (unknown topics will remain NaN)
topics_df["Aspect Label"] = topics_df["Topic"].map(topic_labels)

# Drop rows without a mapped label (optional)
topics_df = topics_df[topics_df["Aspect Label"].notna()]

# Filter out non-response clusters
non_response_labels = [
    "No response (availability)",
    "No response (generic)",
    "No response (time-related)"
]
topics_df = topics_df[~topics_df["Aspect Label"].isin(non_response_labels)]

topics_df.head()

,Course Code,Section Code,Instructor,Semester,Comment Type,Comment Text,Cleaned Comment,Topic,Aspect Label
3,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,Good,good,0,General sentiment (mixed)
4,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,Good,good,0,General sentiment (mixed)
5,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,good,good,0,General sentiment (mixed)
21,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Weaknesses,good,good,0,General sentiment (mixed)
25,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Weaknesses,Excellent,excellent,4,Positive sentiment (teaching quality)


# Then we are preparing the aspect level summary

In [13]:
aspect_summary = topics_df["Aspect Label"].value_counts().reset_index()
aspect_summary.columns = ["Aspect", "Count"]
aspect_summary

,Aspect,Count
0,General sentiment (mixed),91
1,Positive sentiment (teaching quality),13


In [14]:
# Ok so we are going to create a sentiment column for the topics now
def classify_sentiment(text):
    text = str(text).lower()
    positive_words = ["good", "excellent", "ok", "fair", "helpful", "clear", "organized", "great", "amazing", "well done"]
    negative_words = ["poor", "confusing", "bad", "none", "n/a", "nil", "hard", "late"]

    if any(word in text for word in positive_words):
        return "Positive"
    elif any(word in text for word in negative_words):
        return "Negative"
    else:
        return "Neutral"

# Apply to your DataFrame (use the existing topics_df)
topics_df["Sentiment"] = topics_df["Cleaned Comment"].apply(classify_sentiment)

# Preview
topics_df[["Cleaned Comment", "Sentiment"]].head(20)

# Then we are going to verify it 
topics_df["Sentiment"].value_counts()

Sentiment
Positive    100
Negative      4
Name: count, dtype: int64

# Aspect level extension

In [15]:
# Group by aspect and sentiment
aspect_sentiment_summary = (
    topics_df.groupby(["Aspect Label", "Sentiment"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Add a total column (sum only numeric columns to avoid concatenating strings)
numeric_cols = aspect_sentiment_summary.select_dtypes(include=["number"]).columns
aspect_sentiment_summary["Total"] = aspect_sentiment_summary[numeric_cols].sum(axis=1)

# Sort by total descending
aspect_sentiment_summary = aspect_sentiment_summary.sort_values("Total", ascending=False)

aspect_sentiment_summary

Sentiment,Aspect Label,Negative,Positive,Total
0,General sentiment (mixed),4,87,91
1,Positive sentiment (teaching quality),0,13,13


Ok so that works, but let me extend it to also registar negation words before good like not or no so that it doesnt misclassify things

In [16]:
def classify_sentiment(text):
    text = str(text).lower()
    
    positive_words = ["good", "excellent", "ok", "fair", "helpful", "clear", "organized", "great", "amazing", "well done"]
    negative_words = ["poor", "confusing", "bad", "none", "n/a", "nil", "hard", "late"]
    
    # Handle negations explicitly
    negation_patterns = [
        "not good", "not helpful", "not clear", "not fair", "not organized", "not ok", "not great", "not amazing"
    ]
    
    # Check for negation first
    if any(pattern in text for pattern in negation_patterns):
        return "Negative"
    
    # Then check normal positive/negative words
    if any(word in text for word in positive_words):
        return "Positive"
    elif any(word in text for word in negative_words):
        return "Negative"
    else:
        return "Neutral"

# Apply to your DataFrame
topics_df["Sentiment"] = topics_df["Cleaned Comment"].apply(classify_sentiment)

# Preview distribution
topics_df["Sentiment"].value_counts()

Sentiment
Positive    100
Negative      4
Name: count, dtype: int64

ok so that works. Now i need a repsentative quotes for the AI to choose when giving an example in the report

# Representative qouting

In [17]:
# Group by aspect and sentiment
grouped = topics_df.groupby(["Aspect Label", "Sentiment"])

# Extract top 3 comments per group
representative_quotes = grouped["Cleaned Comment"].apply(lambda x: x.head(3)).reset_index()

# Preview
representative_quotes.head(10)

,Aspect Label,Sentiment,level_2,Cleaned Comment
0,General sentiment (mixed),Negative,66,hard but
1,General sentiment (mixed),Negative,69,poor
2,General sentiment (mixed),Negative,204,poor
3,General sentiment (mixed),Positive,3,good
4,General sentiment (mixed),Positive,4,good
5,General sentiment (mixed),Positive,5,good
6,Positive sentiment (teaching quality),Positive,25,excellent
7,Positive sentiment (teaching quality),Positive,51,excellent
8,Positive sentiment (teaching quality),Positive,70,well done amazing


In [18]:
# Here is a little extension, in a readable block format
for (aspect, sentiment), group in grouped:
    print(f"\nAspect: {aspect} | Sentiment: {sentiment}")
    for comment in group.head(3)["Cleaned Comment"]:
        print(f"- {comment}")

# I should come back to this later to see if ill change it or not

# So this is the way to go about it. I wont be messing with this again. its just a matter of refinement next


Aspect: General sentiment (mixed) | Sentiment: Negative
- hard but
- poor
- poor

Aspect: General sentiment (mixed) | Sentiment: Positive
- good
- good
- good

Aspect: Positive sentiment (teaching quality) | Sentiment: Positive
- excellent
- excellent
- well done amazing


ok so this seems alot more better thant he spreed sheet table that we got before. So lets refine it a bit. I dont want duplication of statements. Im also thinkng of a threshold for sentiment and tagging for quotes. Since we ahave a lot of basic ones

# Refinements

In [19]:
# So we are stargting with confidence scoring now
# Confidence = frequency / total comments
total_comments = len(topics_df)

aspect_confidence = (
    topics_df["Aspect Label"].value_counts()
    .apply(lambda x: round(x / total_comments, 2))
    .reset_index()
)

aspect_confidence.columns = ["Aspect", "Confidence"]
aspect_confidence

# This means that a confidence of 0.10 means the aspect was mentioned in 10% of all comments


,Aspect,Confidence
0,General sentiment (mixed),0.88
1,Positive sentiment (teaching quality),0.12


In [20]:
# Next we deal with quotes 
# Deduplicate quotes within each aspect + sentiment group
grouped = topics_df.groupby(["Aspect Label", "Sentiment"])

dedup_quotes = grouped["Cleaned Comment"].apply(lambda x: list(set(x.head(10)))).reset_index()

# Preview
dedup_quotes.head()
# Atleast now good will only appear once in the representative quotes. Studnets are really anooying with these respeonses of theres

,Aspect Label,Sentiment,Cleaned Comment
0,General sentiment (mixed),Negative,"[hard but, poor]"
1,General sentiment (mixed),Positive,[good]
2,Positive sentiment (teaching quality),Positive,"[great, well done amazing, excellent]"


In [21]:
# Laslty is tagging quotes 
def tag_quote(comment, sentiment):
    if sentiment == "Positive":
        return "Praise"
    elif sentiment == "Negative":
        return "Critique"
    else:
        return "Suggestion"

topics_df["Quote Tag"] = topics_df.apply(lambda row: tag_quote(row["Cleaned Comment"], row["Sentiment"]), axis=1)

# Preview
topics_df[["Aspect Label", "Sentiment", "Cleaned Comment", "Quote Tag"]].head(15)

# Sp now we added tags to the quotes for easier identification


,Aspect Label,Sentiment,Cleaned Comment,Quote Tag
3,General sentiment (mixed),Positive,good,Praise
4,General sentiment (mixed),Positive,good,Praise
5,General sentiment (mixed),Positive,good,Praise
21,General sentiment (mixed),Positive,good,Praise
25,Positive sentiment (teaching quality),Positive,excellent,Praise
39,General sentiment (mixed),Positive,good,Praise
40,General sentiment (mixed),Positive,good,Praise
46,General sentiment (mixed),Positive,good,Praise
51,Positive sentiment (teaching quality),Positive,excellent,Praise
52,General sentiment (mixed),Positive,good,Praise


In [22]:
# OK OK OK lets now combine everything together and see what we get now
# Step 1: Confidence scoring
total_comments = len(topics_df)
aspect_confidence = (
    topics_df["Aspect Label"].value_counts()
    .apply(lambda x: round(x / total_comments, 2))
    .to_dict()
)

# Step 2: Deduplicate quotes within aspect + sentiment
grouped = topics_df.groupby(["Aspect Label", "Sentiment"])

# Step 3: Tagging function
def tag_quote(comment, sentiment):
    if sentiment == "Positive":
        return "Praise"
    elif sentiment == "Negative":
        return "Critique"
    else:
        return "Suggestion"

# Step 4: Pivot-style reporting with refinements
for (aspect, sentiment), group in grouped:
    confidence = aspect_confidence.get(aspect, 0)
    print(f"\nAspect: {aspect} | Sentiment: {sentiment} | Confidence: {confidence:.2f}")
    
    # Deduplicate and tag quotes
    quotes = list(set(group["Cleaned Comment"].head(10)))
    for q in quotes:
        tag = tag_quote(q, sentiment)
        print(f"- ({tag}) {q}")


Aspect: General sentiment (mixed) | Sentiment: Negative | Confidence: 0.88
- (Critique) hard but
- (Critique) poor

Aspect: General sentiment (mixed) | Sentiment: Positive | Confidence: 0.88
- (Praise) good

Aspect: Positive sentiment (teaching quality) | Sentiment: Positive | Confidence: 0.12
- (Praise) great
- (Praise) well done amazing
- (Praise) excellent


ok so it works fantastic. And im starting to see my vision come true. But some how im still getting bleed through of the quote 'improve hisher teaching in this course'. Which i had explicitly stated to  be ignored. But yet here it still is.
Well that now needs fixing. 

Alright made the changes in the code and now the pdf we are using should be updated. So lets rerun and see how things look like

Ok so things have been fixed. Now its onto automative narrative generation.
so the plan is this basically:
For each aspect, we’ll:
Summarize the sentiment distribution (e.g., mostly positive, mixed, etc.)
Weave in tagged quotes (Praise, Suggestion, Critique)
Use natural transitions to create flow and readabilit


# Auto-Generate Narrative Paragraphs

In [23]:
from collections import defaultdict

# Group quotes by aspect and sentiment
grouped = topics_df.groupby(["Aspect Label", "Sentiment"])

# Build quote bank with tags
quote_bank = defaultdict(lambda: defaultdict(list))
for (aspect, sentiment), group in grouped:
    quotes = list(set(group["Cleaned Comment"].head(5)))
    for q in quotes:
        tag = tag_quote(q, sentiment)
        quote_bank[aspect][sentiment].append((tag, q))

# Generate narrative paragraphs
for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    print(f"\n Aspect: {aspect}")
    print(f"Summary: Based on {total} comments, this aspect received {pos} positive, {neg} negative, and {neu} neutral remarks.")

    # Positive quotes
    if "Positive" in sentiments:
        print("Students offered praise such as:")
        for tag, quote in sentiments["Positive"]:
            print(f"– {quote}")

    # Negative quotes
    if "Negative" in sentiments:
        print("However, critiques included:")
        for tag, quote in sentiments["Negative"]:
            print(f"– {quote}")

    # Neutral quotes
    if "Neutral" in sentiments:
        print("Suggestions for improvement included:")
        for tag, quote in sentiments["Neutral"]:
            print(f"– {quote}")


 Aspect: General sentiment (mixed)
Summary: Based on 3 comments, this aspect received 1 positive, 2 negative, and 0 neutral remarks.
Students offered praise such as:
– good
However, critiques included:
– hard but
– poor

 Aspect: Positive sentiment (teaching quality)
Summary: Based on 3 comments, this aspect received 3 positive, 0 negative, and 0 neutral remarks.
Students offered praise such as:
– great
– well done amazing
– excellent


OK this is now something. Its a bit bland and boring to read. But that should be something i should be able to improve.

# Adjustment

In [24]:
for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    print(f"\nAspect: {aspect}")
    print(f"This theme was mentioned in {total} comments, with {pos} positive, {neg} negative, and {neu} neutral remarks.")

    if "Positive" in sentiments:
        quotes = [q for _, q in sentiments["Positive"]]
        if quotes:
            print("Students offered praise, for example:")
            for q in quotes[:3]:
                print(f"“{q}”")

    if "Negative" in sentiments:
        quotes = [q for _, q in sentiments["Negative"]]
        if quotes:
            print("Critiques were also noted, such as:")
            for q in quotes[:3]:
                print(f"“{q}”")

    if "Neutral" in sentiments:
        quotes = [q for _, q in sentiments["Neutral"]]
        if quotes:
            print("Suggestions for improvement included:")
            for q in quotes[:3]:
                print(f"“{q}”")


Aspect: General sentiment (mixed)
This theme was mentioned in 3 comments, with 1 positive, 2 negative, and 0 neutral remarks.
Students offered praise, for example:
“good”
Critiques were also noted, such as:
“hard but”
“poor”

Aspect: Positive sentiment (teaching quality)
This theme was mentioned in 3 comments, with 3 positive, 0 negative, and 0 neutral remarks.
Students offered praise, for example:
“great”
“well done amazing”
“excellent”


ok thats better. but we can do a little bit more. Then from there refinement will come from feedback.
So im thinking we can change the opening phrases with a few default ones

# Other Adjustments

In [25]:
import random

# Variation templates
openers = [
    "This theme emerged in {total} comments, reflecting {pos} positive, {neg} negative, and {neu} neutral remarks.",
    "Students frequently mentioned this aspect, with {total} remarks recorded ({pos} positive, {neg} negative, {neu} neutral).",
    "Feedback on this area was mixed, with {total} comments highlighting {pos} positive, {neg} negative, and {neu} neutral points."
]

praise_transitions = [
    "Positive remarks included:",
    "Students offered praise such as:",
    "Examples of appreciation were:"
]

critique_transitions = [
    "However, critiques were also noted:",
    "Concerns raised by students included:",
    "Critical feedback pointed to:"
]

suggestion_transitions = [
    "Suggestions for improvement focused on:",
    "Students recommended adjustments such as:",
    "Ideas for enhancement included:"
]

# Narrative generator with variation
for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    print(f"\n Aspect: {aspect}")
    opener = random.choice(openers)
    print(opener.format(total=total, pos=pos, neg=neg, neu=neu))

    if "Positive" in sentiments:
        quotes = [q for _, q in sentiments["Positive"]]
        if quotes:
            print(random.choice(praise_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")

    if "Negative" in sentiments:
        quotes = [q for _, q in sentiments["Negative"]]
        if quotes:
            print(random.choice(critique_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")

    if "Neutral" in sentiments:
        quotes = [q for _, q in sentiments["Neutral"]]
        if quotes:
            print(random.choice(suggestion_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")


 Aspect: General sentiment (mixed)
Students frequently mentioned this aspect, with 3 remarks recorded (1 positive, 2 negative, 0 neutral).
Examples of appreciation were:
“good”
Concerns raised by students included:
“hard but”
“poor”

 Aspect: Positive sentiment (teaching quality)
This theme emerged in 3 comments, reflecting 3 positive, 0 negative, and 0 neutral remarks.
Examples of appreciation were:
“great”
“well done amazing”
“excellent”


ok so now i want to refine things a bit. With a confidence level. So that the selected text are from a majority of the students feedback

In [26]:
import random
# Threshold for inclusion (e.g., 5%)
CONFIDENCE_THRESHOLD = 0.01


# Variation templates
openers = [
    "This theme emerged in {total} comments, reflecting {pos} positive, {neg} negative, and {neu} neutral remarks.",
    "Students frequently mentioned this aspect, with {total} remarks recorded ({pos} positive, {neg} negative, {neu} neutral).",
    "Feedback on this area was mixed, with {total} comments highlighting {pos} positive, {neg} negative, and {neu} neutral points."
]

praise_transitions = [
    "Positive remarks included:",
    "Students offered praise such as:",
    "Examples of appreciation were:"
]

critique_transitions = [
    "However, critiques were also noted:",
    "Concerns raised by students included:",
    "Critical feedback pointed to:"
]

suggestion_transitions = [
    "Suggestions for improvement focused on:",
    "Students recommended adjustments such as:",
    "Ideas for enhancement included:"
]

# Narrative generator with confidence filter
for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    confidence = total / len(topics_df)

    # Skip aspects below threshold
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    print(f"\n Aspect: {aspect}")
    opener = random.choice(openers)
    print(opener.format(total=total, pos=pos, neg=neg, neu=neu))

    if "Positive" in sentiments:
        quotes = [q for _, q in sentiments["Positive"]]
        if quotes:
            print(random.choice(praise_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")

    if "Negative" in sentiments:
        quotes = [q for _, q in sentiments["Negative"]]
        if quotes:
            print(random.choice(critique_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")

    if "Neutral" in sentiments:
        quotes = [q for _, q in sentiments["Neutral"]]
        if quotes:
            print(random.choice(suggestion_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")


 Aspect: General sentiment (mixed)
Feedback on this area was mixed, with 3 comments highlighting 1 positive, 2 negative, and 0 neutral points.
Students offered praise such as:
“good”
Critical feedback pointed to:
“hard but”
“poor”

 Aspect: Positive sentiment (teaching quality)
This theme emerged in 3 comments, reflecting 3 positive, 0 negative, and 0 neutral remarks.
Examples of appreciation were:
“great”
“well done amazing”
“excellent”


ok so i want it to now notice emerging themes and trends. so readers immediately know whether an aspect is strongly supported or just beginning to surface?

In [27]:
import random

print("Starting narrative generation...")

print(f"Total aspects in quote_bank: {len(quote_bank)}")

for aspect in quote_bank:
    print(f"Processing aspect: {aspect}")
    # Add a break to confirm loop is entered
    break

print(len(quote_bank))  # Should be > 0
CONFIDENCE_THRESHOLD = 0.01

def confidence_label(confidence):
    if confidence >= 0.10:
        return "High confidence theme"
    elif confidence >= 0.05:
        return "Emerging theme"
    else:
        return "Low confidence theme"

# Narrative generator with confidence annotation
for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    confidence = total / len(topics_df)
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    label = confidence_label(confidence)

    print(f"\n Aspect: {aspect} ({label})")
    opener = random.choice(openers)
    print(opener.format(total=total, pos=pos, neg=neg, neu=neu))

    if "Positive" in sentiments:
        quotes = [q for _, q in sentiments["Positive"]]
        if quotes:
            print(random.choice(praise_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")

    if "Negative" in sentiments:
        quotes = [q for _, q in sentiments["Negative"]]
        if quotes:
            print(random.choice(critique_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")

    if "Neutral" in sentiments:
        quotes = [q for _, q in sentiments["Neutral"]]
        if quotes:
            print(random.choice(suggestion_transitions))
            for q in quotes[:3]:
                print(f"“{q}”")


Starting narrative generation...
Total aspects in quote_bank: 2
Processing aspect: General sentiment (mixed)
2

 Aspect: General sentiment (mixed) (Low confidence theme)
Students frequently mentioned this aspect, with 3 remarks recorded (1 positive, 2 negative, 0 neutral).
Examples of appreciation were:
“good”
Critical feedback pointed to:
“hard but”
“poor”

 Aspect: Positive sentiment (teaching quality) (Low confidence theme)
This theme emerged in 3 comments, reflecting 3 positive, 0 negative, and 0 neutral remarks.
Positive remarks included:
“great”
“well done amazing”
“excellent”


ok so it seems that we have 2 issues here, one being the small test size and secodnly being the confidence threshold. Because when we run it at 0.05, theres nothing but when we shift it to 0.01 there is something. So scale is a factor here. But i do not want to be using the complete dataset cause my laptop is slow af and will have run times of like an hour +. So i gotta figure out how to handle this

# handling

so this is the plan basically
lower confidence threshold
Set CONFIDENCE_THRESHOLD =  0.0 for full preview. This ensures all aspects show up, even if they’re weak signals.
Expand representative quotes
Instead of limiting to 3 quotes, allow up to 8–10 per aspect. This gives richer context and simulates what you’ll see with larger datasets.


In [28]:
import random

CONFIDENCE_THRESHOLD = 0.00   # very low for simulation
MAX_QUOTES = 8                # expand quotes per aspect

def confidence_label(confidence):
    if confidence >= 0.10:
        return "High confidence theme"
    elif confidence >= 0.05:
        return "Emerging theme"
    else:
        return "Low confidence theme"

for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    confidence = total / len(topics_df)
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    label = confidence_label(confidence)

    print(f"\n Aspect: {aspect} ({label})")
    opener = random.choice(openers)
    print(opener.format(total=total, pos=pos, neg=neg, neu=neu))

    if "Positive" in sentiments:
        quotes = [q for _, q in sentiments["Positive"]]
        if quotes:
            print(random.choice(praise_transitions))
            for q in quotes[:MAX_QUOTES]:
                print(f"“{q}”")

    if "Negative" in sentiments:
        quotes = [q for _, q in sentiments["Negative"]]
        if quotes:
            print(random.choice(critique_transitions))
            for q in quotes[:MAX_QUOTES]:
                print(f"“{q}”")

    if "Neutral" in sentiments:
        quotes = [q for _, q in sentiments["Neutral"]]
        if quotes:
            print(random.choice(suggestion_transitions))
            for q in quotes[:MAX_QUOTES]:
                print(f"“{q}”")

    # Add a summary sentence at the end
    print(f"Overall, this aspect is classified as a {label}, with feedback balancing praise, critique, and suggestions.")


 Aspect: General sentiment (mixed) (Low confidence theme)
This theme emerged in 3 comments, reflecting 1 positive, 2 negative, and 0 neutral remarks.
Positive remarks included:
“good”
Concerns raised by students included:
“hard but”
“poor”
Overall, this aspect is classified as a Low confidence theme, with feedback balancing praise, critique, and suggestions.

 Aspect: Positive sentiment (teaching quality) (Low confidence theme)
Students frequently mentioned this aspect, with 3 remarks recorded (3 positive, 0 negative, 0 neutral).
Students offered praise such as:
“great”
“well done amazing”
“excellent”
Overall, this aspect is classified as a Low confidence theme, with feedback balancing praise, critique, and suggestions.


ok this is now something. let me now randomize the closing statement so that it feels unique. We’ll rotate through a set of templates, just like we did for openers and transitions. 

In [29]:
import random

CONFIDENCE_THRESHOLD = 0.00   # very low for simulation
MAX_QUOTES = 8                # expand quotes per aspect

def confidence_label(confidence):
    if confidence >= 0.10:
        return "High confidence theme"
    elif confidence >= 0.05:
        return "Emerging theme"
    else:
        return "Low confidence theme"

for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    confidence = total / len(topics_df)
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    label = confidence_label(confidence)

    print(f"\n Aspect: {aspect} ({label})")
    opener = random.choice(openers)
    print(opener.format(total=total, pos=pos, neg=neg, neu=neu))

    if "Positive" in sentiments:
        quotes = [q for _, q in sentiments["Positive"]]
        if quotes:
            print(random.choice(praise_transitions))
            for q in quotes[:MAX_QUOTES]:
                print(f"“{q}”")

    if "Negative" in sentiments:
        quotes = [q for _, q in sentiments["Negative"]]
        if quotes:
            print(random.choice(critique_transitions))
            for q in quotes[:MAX_QUOTES]:
                print(f"“{q}”")

    if "Neutral" in sentiments:
        quotes = [q for _, q in sentiments["Neutral"]]
        if quotes:
            print(random.choice(suggestion_transitions))
            for q in quotes[:MAX_QUOTES]:
                print(f"“{q}”")
    closing_templates = {
    "High confidence theme": [
        "Overall, this aspect stands out as a high confidence theme, with clear strengths and actionable suggestions.",
        "Taken together, the feedback positions this as a well‑established theme with consistent signals.",
        "This aspect reflects a strong consensus, balancing praise with constructive critique."
    ],
    "Emerging theme": [
        "Overall, this aspect is emerging, with early signals pointing to areas worth monitoring.",
        "The feedback suggests this is a developing theme, offering useful but less frequent insights.",
        "This aspect is beginning to surface, with comments highlighting opportunities for refinement."
    ],
    "Low confidence theme": [
        "Overall, this aspect appears as a low confidence theme, with limited but noteworthy remarks.",
        "The feedback here is sparse, suggesting only minor signals at this stage.",
        "This aspect is weakly represented, with comments offering isolated perspectives."
    ]
}

# At the end of each aspect block:
closing = random.choice(closing_templates[label])
print(closing)


 Aspect: General sentiment (mixed) (Low confidence theme)
Students frequently mentioned this aspect, with 3 remarks recorded (1 positive, 2 negative, 0 neutral).
Positive remarks included:
“good”
Concerns raised by students included:
“hard but”
“poor”

 Aspect: Positive sentiment (teaching quality) (Low confidence theme)
Students frequently mentioned this aspect, with 3 remarks recorded (3 positive, 0 negative, 0 neutral).
Students offered praise such as:
“great”
“well done amazing”
“excellent”
Overall, this aspect appears as a low confidence theme, with limited but noteworthy remarks.


ok ive ran it a couple of times to see the different outputs come together. Now i just need to balance out how many quotes we get for each aspect. since its dumping everything that available

In [30]:
import random

CONFIDENCE_THRESHOLD = 0.01   # keep low for testing
MAX_QUOTES_PER_SENTIMENT = 3  # balanced cap

def confidence_label(confidence):
    if confidence >= 0.10:
        return "High confidence theme"
    elif confidence >= 0.05:
        return "Emerging theme"
    else:
        return "Low confidence theme"

for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    confidence = total / len(topics_df)
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    label = confidence_label(confidence)

    print(f"\n Aspect: {aspect} ({label})")
    opener = random.choice(openers)
    print(opener.format(total=total, pos=pos, neg=neg, neu=neu))

    if "Positive" in sentiments:
        quotes = [q for _, q in sentiments["Positive"]]
        if quotes:
            print(random.choice(praise_transitions))
            for q in random.sample(quotes, min(len(quotes), MAX_QUOTES_PER_SENTIMENT)):
                print(f"“{q}”")

    if "Negative" in sentiments:
        quotes = [q for _, q in sentiments["Negative"]]
        if quotes:
            print(random.choice(critique_transitions))
            for q in random.sample(quotes, min(len(quotes), MAX_QUOTES_PER_SENTIMENT)):
                print(f"“{q}”")

    if "Neutral" in sentiments:
        quotes = [q for _, q in sentiments["Neutral"]]
        if quotes:
            print(random.choice(suggestion_transitions))
            for q in random.sample(quotes, min(len(quotes), MAX_QUOTES_PER_SENTIMENT)):
                print(f"“{q}”")

    # Varied closing summary
    closing = random.choice(closing_templates[label])
    print(closing)


 Aspect: General sentiment (mixed) (Low confidence theme)
Feedback on this area was mixed, with 3 comments highlighting 1 positive, 2 negative, and 0 neutral points.
Students offered praise such as:
“good”
However, critiques were also noted:
“hard but”
“poor”
This aspect is weakly represented, with comments offering isolated perspectives.

 Aspect: Positive sentiment (teaching quality) (Low confidence theme)
Students frequently mentioned this aspect, with 3 remarks recorded (3 positive, 0 negative, 0 neutral).
Students offered praise such as:
“great”
“well done amazing”
“excellent”
Overall, this aspect appears as a low confidence theme, with limited but noteworthy remarks.


ok so we have a good base foundation here. Now we need to start building it to match the sample out that we are aiming for.

In [31]:
import random

CONFIDENCE_THRESHOLD = 0.00
MAX_QUOTES_PER_SENTIMENT = 3

closing_templates = {
    "High confidence theme": [
        "Overall, this aspect stands out as a high confidence theme, with clear strengths and actionable suggestions.",
        "Taken together, the feedback positions this as a well‑established theme with consistent signals.",
        "This aspect reflects a strong consensus, balancing praise with constructive critique."
    ],
    "Emerging theme": [
        "Overall, this aspect is emerging, with early signals pointing to areas worth monitoring.",
        "The feedback suggests this is a developing theme, offering useful but less frequent insights.",
        "This aspect is beginning to surface, with comments highlighting opportunities for refinement."
    ],
    "Low confidence theme": [
        "Overall, this aspect appears as a low confidence theme, with limited but noteworthy remarks.",
        "The feedback here is sparse, suggesting only minor signals at this stage.",
        "This aspect is weakly represented, with comments offering isolated perspectives."
    ]
}

def confidence_label(confidence):
    if confidence >= 0.10:
        return "High confidence theme"
    elif confidence >= 0.05:
        return "Emerging theme"
    else:
        return "Low confidence theme"

for aspect, sentiments in quote_bank.items():
    total = sum(len(quotes) for quotes in sentiments.values())
    pos = len(sentiments.get("Positive", []))
    neg = len(sentiments.get("Negative", []))
    neu = len(sentiments.get("Neutral", []))

    confidence = total / len(topics_df)
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    label = confidence_label(confidence)

    # Percentages
    pos_pct = round((pos / total) * 100, 1) if total > 0 else 0
    neg_pct = round((neg / total) * 100, 1) if total > 0 else 0
    neu_pct = round((neu / total) * 100, 1) if total > 0 else 0

    print(f"\n Aspect: {aspect} ({label})")

    # Overall sentiment section
    print(f"Overall Sentiment:\nPositive: {pos_pct}% | Neutral: {neu_pct}% | Negative: {neg_pct}%")

    # Strengths
    if "Positive" in sentiments and sentiments["Positive"]:
        quotes = [q for _, q in sentiments["Positive"]]
        print("\nWhat Students Consistently Praised:")
        for q in random.sample(quotes, min(len(quotes), MAX_QUOTES_PER_SENTIMENT)):
            print(f"“{q}”")

    # Suggestions
    if "Neutral" in sentiments and sentiments["Neutral"]:
        quotes = [q for _, q in sentiments["Neutral"]]
        print("\nCommon Suggestions for Improvement:")
        for q in random.sample(quotes, min(len(quotes), MAX_QUOTES_PER_SENTIMENT)):
            print(f"“{q}”")

    # Critiques
    if "Negative" in sentiments and sentiments["Negative"]:
        quotes = [q for _, q in sentiments["Negative"]]
        print("\nCritiques or Concerns Raised:")
        for q in random.sample(quotes, min(len(quotes), MAX_QUOTES_PER_SENTIMENT)):
            print(f"“{q}”")

    # Student voice (pick one representative quote across all sentiments)
    all_quotes = []
    for s in sentiments.values():
        all_quotes.extend([q for _, q in s])
    if all_quotes:
        student_voice = random.choice(all_quotes)
        print("\nExample Student Voice:")
        print(f"“{student_voice}”")

    # Closing summary
    closing = random.choice(closing_templates[label])
    print(f"\nAI Summary in Plain Language:\n{closing}")


 Aspect: General sentiment (mixed) (Low confidence theme)
Overall Sentiment:
Positive: 33.3% | Neutral: 0.0% | Negative: 66.7%

What Students Consistently Praised:
“good”

Critiques or Concerns Raised:
“poor”
“hard but”

Example Student Voice:
“good”

AI Summary in Plain Language:
This aspect is weakly represented, with comments offering isolated perspectives.

 Aspect: Positive sentiment (teaching quality) (Low confidence theme)
Overall Sentiment:
Positive: 100.0% | Neutral: 0.0% | Negative: 0.0%

What Students Consistently Praised:
“well done amazing”
“great”
“excellent”

Example Student Voice:
“excellent”

AI Summary in Plain Language:
This aspect is weakly represented, with comments offering isolated perspectives.


ok so thats looking sharper. let’s build the micro‑level aggregation logic so your aspect‑level narratives roll up into a polished lecturer report 


# Micro level aggregation

In [ ]:
import random
from collections import Counter

def generate_micro_report_md(df_course, quote_bank):
    # 1) Auto metadata from parsed comments df (single course/section subset)
    course_code = topics_df['Course Code'].iloc[0] if 'Course Code' in topics_df else "Unknown Course"
    section_code = topics_df['Section Code'].iloc[0] if 'Section Code' in topics_df else ""
    lecturer_name = topics_df['Instructor'].iloc[0] if 'Instructor' in topics_df else "Unknown Lecturer"
    semester = topics_df['Semester'].iloc[0] if 'Semester' in topics_df else "Semester not captured"
    responses = len(topics_df)

    # 2) Sentiment totals from quote_bank
    total_comments = sum(sum(len(v) for v in sentiments.values()) for sentiments in quote_bank.values()) or max(responses, 1)

    pos_total = sum(len(sentiments.get("Positive", [])) for sentiments in quote_bank.values())
    neg_total = sum(len(sentiments.get("Negative", [])) for sentiments in quote_bank.values())
    neu_total = sum(len(sentiments.get("Neutral", [])) for sentiments in quote_bank.values())

    pos_pct = round((pos_total / total_comments) * 100, 1) if total_comments else 0
    neu_pct = round((neu_total / total_comments) * 100, 1) if total_comments else 0
    neg_pct = round((neg_total / total_comments) * 100, 1) if total_comments else 0

    # 3) Aggregate themes
    praises = []
    suggestions = []

    for sentiments in quote_bank.values():
        praises.extend([q for _, q in sentiments.get("Positive", [])])
        suggestions.extend([q for _, q in sentiments.get("Neutral", [])])

    praise_counts = Counter(praises)
    suggestion_counts = Counter(suggestions)

    top_praises = praise_counts.most_common(3)
    top_suggestions = suggestion_counts.most_common(3)

    # 4) Build markdown
    lines = []
    lines.append("Lecturer Feedback Report — Micro Level")
    lines.append(f"Course: {course_code} – Section {section_code}")
    lines.append(f"Lecturer: {lecturer_name}")
    lines.append(f"Semester: {semester}")
    lines.append(f"Responses: {responses} students")
    lines.append("")
    lines.append("1️ Overall sentiment")
    lines.append(f"Open‑ended sentiment: Positive: {pos_pct}% | Neutral: {neu_pct}% | Negative: {neg_pct}%")
    lines.append("")

    lines.append("2️ What students consistently praised")
    if top_praises:
        for theme, count in top_praises:
            pct = round((count / max(total_comments, 1)) * 100, 1)
            lines.append(f"- {theme} — {pct}% of positive comments")
        if praises:
            lines.append("")
            lines.append(f"Example student voice: “{random.choice(praises)}”")
    else:
        lines.append("- No explicit praise captured in this subset.")
    lines.append("")

    lines.append("3️ Common suggestions for improvement")
    if top_suggestions:
        for theme, count in top_suggestions:
            pct = round((count / max(total_comments, 1)) * 100, 1)
            lines.append(f"- {theme} — {pct}% of suggestions")
        if suggestions:
            lines.append("")
            lines.append(f"Example student voice: “{random.choice(suggestions)}”")
    else:
        lines.append("- No explicit suggestions captured in this subset.")
    lines.append("")

    lines.append("5️ Recommended actions")
    if top_suggestions:
        for theme, _ in top_suggestions:
            lines.append(f"- {theme}")
    else:
        lines.append("- No actions inferred due to limited suggestions.")
    lines.append("")
    # this is a just place holder for until i actually build the AI then it generate its own
    lines.append("6️ AI summary in plain language")
    lines.append("“Students value your delivery and clarity. The main opportunities are adding practical elements, "
                 "adjusting pacing, and posting materials earlier. Addressing these should strengthen engagement and satisfaction.”")

    return "\n".join(lines)

print(generate_micro_report_md(topics_df, quote_bank))

Lecturer Feedback Report — Micro Level
Course: IST3005 – Section A
Lecturer: Afundi, Patrick
Semester: Summer Semester 2019 (UNDG)
Responses: 104 students

1️ Overall sentiment
Open‑ended sentiment: Positive: 66.7% | Neutral: 0.0% | Negative: 33.3%

2️ What students consistently praised
- good — 16.7% of positive comments
- great — 16.7% of positive comments
- well done amazing — 16.7% of positive comments

Example student voice: “well done amazing”

3️ Common suggestions for improvement
- No explicit suggestions captured in this subset.

5️ Recommended actions
- No actions inferred due to limited suggestions.

6️ AI summary in plain language
“Students value your delivery and clarity. The main opportunities are adding practical elements, adjusting pacing, and posting materials earlier. Addressing these should strengthen engagement and satisfaction.”


again i hate this life. I just want a farm. Ive just realised we did not take into account getting the semester year included in so that when i generate the report its already present. So that means its another journey back to the parser code.

glad the updated changes work now. And we are getting the structure that i need.